# E. coli Glycolytic Network Construction
## Growth Medium: Glucose
Growth data obtained from the following sources: 
- Gerosa, Luca et al. “Pseudo-transition Analysis Identifies the Key Regulators of Dynamic Metabolic Adaptations from Steady-State Data.” Cell systems vol. 1,4 (2015): 270-82. doi:10.1016/j.cels.2015.09.008


- Volkmer, Benjamin, and Matthias Heinemann. “Condition-dependent cell volume and concentration of Escherichia coli to facilitate data conversion for systems biology modeling.” PloS one vol. 6,7 (2011): e23126. doi:10.1371/journal.pone.0023126



### Import packages

In [1]:
# Disable gurobi logging output for this notebook.
try:
    import gurobipy
    gurobipy.setParam("OutputFlag", 0)
except ImportError:
    pass


import numpy as np
import pandas as pd
import sympy as sym


import cobra
from cobra.io.json import load_json_model as load_cobra_json_model

import mass
from mass import MassConfiguration, MassModel, MassMetabolite, MassReaction, Simulation
from mass.io.json import save_json_model as save_mass_json_model

print(f"COBRApy version: {cobra.__version__}")
print(f"MASSpy version: {mass.__version__}")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-21
COBRApy version: 0.22.1
MASSpy version: 0.1.5


### Set configuration and paths

In [2]:
MASSCONFIGURATION = MassConfiguration()
MASSCONFIGURATION.solver = "gurobi"

## Load COBRA model

In [3]:
cobra_model = load_cobra_json_model(f"./models/cobra/iML1515.json")

## Obtain Flux State
### Load growth data

In [4]:
medium = "Glucose"

flux_data = pd.read_excel(
    io="./data/growth_data.xlsx",
    sheet_name="flux_data",
    index_col=0
)
flux_data = flux_data.loc[lambda x: x['Growth Medium'] == medium]
flux_data = flux_data.drop("Growth Medium", axis=1)
flux_data

,Flux (mmol * gDW-1 * h-1)
ID,
EX_ac_e,6.827019e+00
ACt2rpp,-6.827019e+00
ACKr,-6.827019e+00
PTAr,6.827019e+00
ACS,0.000000e+00
...,...
ME1,-4.778621e-08
ME2,-4.778621e-08
ICL,9.654038e-10


### Set bounds
#### Growth rate

In [5]:
biomass_rxn = cobra_model.reactions.BIOMASS_Ec_iML1515_core_75p37M
growth_rate = flux_data.loc[biomass_rxn.id][0]
biomass_rxn.bounds = (growth_rate, growth_rate)
biomass_rxn.bounds

(0.65, 0.65)

#### Glucose Exchange

In [6]:
EX_glc__D_e = cobra_model.reactions.EX_glc__D_e
medium_uptake = flux_data.loc[EX_glc__D_e.id][0]
EX_glc__D_e.bounds = (medium_uptake, 1000)
EX_glc__D_e.bounds

(-9.654, 1000)

### Formulate QP minimization for fluxes

In [7]:
v_vars = []
v_data = []

# For irreversible enzyme pairs, flux data is given as Enzyme1 - Enzyme2 = value.
# To ensure all enzymes have some flux, add a percentage of the net flux for each enzyme
# The netflux will still remain the same value.
reverse_flux_percent = 0.1

irreversible_enzyme_pairs = [["PFK", "FBP"], ["PYK", "PPS"]]


for rid, flux in flux_data.itertuples():
    # Make adjustments to net flux of PFK/FBP and PYK/PPS to ensure
    # no target flux value is 0 in order to create an enzyme module.
    for irreversible_enzyme_pair in irreversible_enzyme_pairs:
        if rid in irreversible_enzyme_pair:
            flux1, flux2 = flux_data.loc[irreversible_enzyme_pair, "Flux (mmol * gDW-1 * h-1)"].values
            if flux1 == 0:
                flux += reverse_flux_percent * flux2 # mmol*gDW^-1*hr^-1
            if flux2 == 0:
                flux += reverse_flux_percent * flux1 # mmol*gDW^-1*hr^-1
            print(rid, flux)
    v_vars.append(sym.Symbol(rid))
    v_data.append(flux)

# Make symbolic for optlang objective 
v_vars = sym.Matrix(v_vars)
v_data = sym.Matrix(v_data)
F = sym.Matrix(2 * sym.eye(len(v_vars)))

objective = 0.5 * v_vars.T * F * v_vars  - (2 * v_data).T * v_vars
cobra_model.objective = objective[0]
cobra_model.objective_direction = "min"

flux_solution = cobra_model.optimize()

PFK 7.76432470140912
FBP 0.7058477001281019
PYK 2.7360389076214697
PPS 0.24873080978377


#### Compare growth data to computed fluxes

In [8]:
from module4_functions import compare_data


In [9]:
compare_data(flux_data, flux_solution.fluxes, scale="linear")


alt.LayerChart(...)

## Create MASS Model

In [10]:
# Create MassModel
mass_model = MassModel("Glycolysis", array_type="DataFrame")

# Reactions to extract into subnetwork
reaction_list = [
    "PGI", 
    "PFK", 
    "FBP", 
    "FBA", 
    "TPI", 
    "GAPD",
    "PGK", 
    "PGM", 
    "ENO", 
    "PYK", 
    "PPS", 
    "LDH_D",
]


cobra_reactions = cobra_model.reactions.get_by_any(reaction_list)
mass_model.add_reactions([MassReaction(rxn) for rxn in cobra_reactions])
mass_model

Name,Glycolysis
Memory address,0x07f9d9970d5b0
Stoichiometric Matrix,19x12
Matrix Rank,12
Number of metabolites,19
Initial conditions defined,0/19
Number of reactions,12
Number of genes,12
Number of enzyme modules,0
Number of groups,0
Objective expression,0


### Convert flux units to M/hr

In [11]:
T = 313.15
gas_constant = 0.008314
e_coli_density = 1.1 # g / mL assumption
volume = 3.2  # femtoliter

# Perform conversions
doubling_time_per_minute = np.log(2) / growth_rate * 60
cell_gDW = 42000 * doubling_time_per_minute**-1.232 * 1e-15
real_cell_total_weight = e_coli_density * (volume * 1e-12) # fL --> mL

# Assume water is 70%
adj_volume = volume * 0.7
gDW_L_conversion_factor = real_cell_total_weight / (adj_volume * 1e-15)

for reaction in mass_model.reactions.get_by_any(reaction_list):
    flux = flux_solution[reaction.id]
    reaction.steady_state_flux = flux * gDW_L_conversion_factor * 0.001

## Set equilibrium constants

In [12]:
Keq_data = pd.read_excel(
    io="./data/growth_data.xlsx",
    sheet_name="Keq_data",
    index_col=0
)

for reaction in mass_model.reactions.get_by_any(reaction_list):
    reaction.Keq = Keq_data.loc[reaction.id][0]

## Obtain Concentration State
### Load growth data

In [13]:
conc_data = pd.read_excel(
    io="./data/growth_data.xlsx",
    sheet_name="conc_data",
    index_col=0
)
conc_data = conc_data.loc[lambda x: x['Growth Medium'] == "Glucose"]
conc_data = conc_data.drop("Growth Medium", axis=1)
conc_data

,Concentration (mol * L-1)
ID,
adp_c,0.002185
amp_c,0.001743
atp_c,0.012466
nad_c,0.007636
nadh_c,0.000099
13dpg_c,0.000184
f6p_c,0.001214
dhap_c,0.004772
g6p_c,0.003431


### Add PFK1 activator GDP

In [14]:
gdp_c = MassMetabolite(cobra_model.metabolites.gdp_c)
# Set the activator as a constant
gdp_c.fixed = True

mass_model.add_metabolites(gdp_c)

### Set initial concentrations from growth data

In [15]:
mass_model.update_initial_conditions({
    mid: value for mid, value in conc_data.itertuples()
})

# Fix hydrogen and water as constants and set concentration to 1.
for metabolite in mass_model.metabolites.get_by_any(["h2o_c", "h_c"]):
    metabolite.fixed = True
    metabolite.initial_condition = 1

missing_ics = mass_model.metabolites.query(lambda m: m.initial_condition is None)
# Provide initial guesses for missing metabolites (pi_c, g3p_c, and lac__D_c)
for metabolite in missing_ics:
    metabolite.initial_condition = 0.001

### Formulate QP minimization for concentrations

In [16]:
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)

In [17]:
conc_solver = ConcSolver(
    mass_model,
    excluded_metabolites=["h_c", "h2o_c"],
    constraint_buffer=1,
    equilibrium_reactions=[x.id for x in mass_model.reactions if x.steady_state_flux == 0]

)

conc_solver.setup_feasible_qp_problem(
    fixed_conc_bounds=list(mass_model.fixed),
    fixed_Keq_bounds=mass_model.reactions.list_attr("Keq_str")
)

conc_solution = conc_solver.optimize()
conc_solution

,variables,reduced_costs
f6p_c,0.002030,0.000000
g6p_c,0.002052,0.000000
adp_c,0.002185,0.000000
atp_c,0.012466,0.000000
fdp_c,0.012562,0.000000
...,...,...
Keq_PGM,5.300000,2.667707
Keq_ENO,5.190000,0.000000
Keq_PYK,21739.130400,0.000000
Keq_PPS,2.410000,0.000000


In [18]:
compare_data(conc_data, conc_solution.concentrations, scale="log")

alt.LayerChart(...)

In [19]:
# Fix Metabolite IDs as SBML compatible before next step
for metabolite in mass_model.metabolites:
    if metabolite.id[0].isdigit():
        metabolite.id = f"_{metabolite.id}"
mass_model.repair()

#### Sample Concentrations

In [20]:
n_models = 10

In [21]:
conc_solver = ConcSolver(
    mass_model,
    excluded_metabolites=["h_c", "h2o_c"],
    constraint_buffer=1,
    equilibrium_reactions=[x.id for x in mass_model.reactions if x.steady_state_flux == 0]
)

conc_solver.setup_sampling_problem(
    fixed_conc_bounds=list(mass_model.fixed),
    fixed_Keq_bounds=mass_model.reactions.list_attr("Keq_str"))
for variable in conc_solver.variables:
    try:
        met = mass_model.metabolites.get_by_id(variable.name)
        variable.lb, variable.ub = np.log([met.ic / 10, met.ic * 10])
    except:
        pass
conc_samples = sample_concentrations(conc_solver, n=n_models, seed=4)
conc_samples

,f6p_c,g6p_c,adp_c,atp_c,fdp_c,pi_c,dhap_c,g3p_c,_13dpg_c,nad_c,nadh_c,_3pg_c,_2pg_c,pep_c,pyr_c,amp_c,lac__D_c,gdp_c
0,0.008653,0.008403,0.002167,0.006164,0.025045,0.003395,0.013169,0.000112,0.000064,0.062893,0.000021,0.013222,0.000647,0.000935,0.000582,0.000213,0.002889,0.000153
1,0.002197,0.001186,0.000799,0.021458,0.040737,0.008274,0.009320,0.000257,0.000589,0.035601,0.000021,0.012665,0.000689,0.000837,0.000831,0.000610,0.007439,0.000136
2,0.000598,0.000429,0.000465,0.004686,0.012635,0.008316,0.006858,0.000108,0.000090,0.023763,0.000012,0.006152,0.000300,0.000215,0.000946,0.001014,0.006923,0.000477
3,0.002264,0.000766,0.001056,0.013164,0.027618,0.005297,0.007487,0.000217,0.000195,0.030322,0.000015,0.007845,0.000310,0.000291,0.001111,0.004797,0.008250,0.001645
4,0.001251,0.001242,0.000873,0.002082,0.011147,0.002912,0.005270,0.000124,0.000056,0.075366,0.000017,0.007944,0.000373,0.000393,0.001096,0.002546,0.003715,0.001255
5,0.001032,0.000864,0.005897,0.012397,0.011884,0.007611,0.004480,0.000156,0.000171,0.066700,0.000012,0.014947,0.000332,0.000494,0.000563,0.000182,0.001465,0.000619
6,0.001139,0.000773,0.001235,0.001675,0.012125,0.004851,0.004392,0.000162,0.000089,0.033457,0.000017,0.015502,0.000471,0.000332,0.000701,0.000206,0.005420,0.000402
7,0.000606,0.000548,0.000818,0.002101,0.033568,0.004490,0.009425,0.000210,0.000062,0.051618,0.000061,0.012479,0.000786,0.000950,0.000565,0.000669,0.009943,0.000493
8,0.002844,0.001676,0.001313,0.002595,0.020228,0.003330,0.007922,0.000150,0.000049,0.046600,0.000047,0.013281,0.000651,0.000282,0.000511,0.000180,0.007717,0.000300
9,0.001875,0.001623,0.000752,0.002716,0.012588,0.004085,0.006728,0.000110,0.000075,0.015972,0.000014,0.011012,0.000653,0.000147,0.000651,0.000185,0.008614,0.000902


In [22]:
models_for_ensemble = []
for idx, conc_sample in conc_samples.iterrows():
    # Make copy of new model
    new_model = mass_model.copy()
    new_model.id += "_C{0:d}".format(idx)
    print(f"Creating model {new_model.id}")
    # Get concentration sample and update model with sample
    new_model.update_initial_conditions(conc_sample.to_dict())
    fluxes = np.array(list(new_model.steady_state_fluxes.values()))
    imbalanced_metabolites = new_model.S.dot(fluxes)
    # Iterate through metabolites
    for mid, imbalance in imbalanced_metabolites.iteritems():
        # Ignore balanced metabolites
        if imbalance == 0:
            continue
        # Get metabolite object
        met = new_model.metabolites.get_by_id(mid)

        # Add boundary reactions for imbalanced metabolites
        boundary_type = "sink"    
        # Add boundary reaction with imbalance as flux value
        boundary_reaction = new_model.add_boundary(
            mid, boundary_type, boundary_condition=met.ic)

        boundary_reaction.Keq = 1
        if imbalance < 0:
            boundary_reaction.reverse_stoichiometry(inplace=True)
            imbalance = -imbalance

        boundary_reaction.kf = imbalance / met.ic
        boundary_reaction.steady_state_flux = imbalance
        try:
            # Update PERCs
            new_model.calculate_PERCs(
                fluxes={
                    r: v for r, v in new_model.steady_state_fluxes.items()
                    if not r.boundary},
                update_reactions=True)
        except:
            print("Negative PERCs for {0}".format(new_model.id))
            continue
    models_for_ensemble.append(new_model)
print("Number of models in ensemble: {0:d}".format(
    len(models_for_ensemble)))

Creating model Glycolysis_C0
Creating model Glycolysis_C1
Creating model Glycolysis_C2
Creating model Glycolysis_C3
Creating model Glycolysis_C4
Creating model Glycolysis_C5
Creating model Glycolysis_C6
Creating model Glycolysis_C7
Creating model Glycolysis_C8
Creating model Glycolysis_C9
Number of models in ensemble: 10


In [23]:
reference_model = models_for_ensemble[0]
sim = Simulation(reference_model)
sim.integrator.maximum_time_step = 100
sim.integrator.absolute_tolerance = 1e-15
sim.integrator.relative_tolerance = 1e-9

tfinal = 1e5
conc_sol, flux_sol = sim.simulate(reference_model, time=(0, tfinal))
df = conc_sol.to_frame()
df

,f6p_c,g6p_c,adp_c,atp_c,fdp_c,pi_c,dhap_c,g3p_c,_13dpg_c,nad_c,nadh_c,_3pg_c,_2pg_c,pep_c,pyr_c,amp_c,lac__D_c
Time,,,,,,,,,,,,,,,,,
0.000000e+00,0.008653,0.008403,0.002167,0.006164,0.025045,0.003395,0.013169,0.000112,0.000064,0.062893,0.000021,0.013222,0.000647,0.000935,0.000582,0.000213,0.002889
2.334948e-11,0.008653,0.008403,0.002167,0.006164,0.025045,0.003395,0.013169,0.000112,0.000064,0.062893,0.000021,0.013222,0.000647,0.000935,0.000582,0.000213,0.002889
4.669897e-11,0.008653,0.008403,0.002167,0.006164,0.025045,0.003395,0.013169,0.000112,0.000064,0.062893,0.000021,0.013222,0.000647,0.000935,0.000582,0.000213,0.002889
7.004845e-11,0.008653,0.008403,0.002167,0.006164,0.025045,0.003395,0.013169,0.000112,0.000064,0.062893,0.000021,0.013222,0.000647,0.000935,0.000582,0.000213,0.002889
1.429908e-10,0.008653,0.008403,0.002167,0.006164,0.025045,0.003395,0.013169,0.000112,0.000064,0.062893,0.000021,0.013222,0.000647,0.000935,0.000582,0.000213,0.002889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.967180e+04,0.004422,0.009726,0.001237,0.008535,0.026469,0.001925,0.010417,0.000243,0.000068,0.032035,0.000031,0.008258,0.000533,0.001421,0.001002,0.000722,0.014569
9.977180e+04,0.004422,0.009726,0.001237,0.008535,0.026469,0.001925,0.010417,0.000243,0.000068,0.032035,0.000031,0.008258,0.000533,0.001421,0.001002,0.000722,0.014569
9.987180e+04,0.004422,0.009726,0.001237,0.008535,0.026469,0.001925,0.010417,0.000243,0.000068,0.032035,0.000031,0.008258,0.000533,0.001421,0.001002,0.000722,0.014569


In [24]:
from module4_functions import plot_time_profile

In [25]:
plot_time_profile(conc_sol, xscale="log", yscale="log")

alt.Chart(...)

In [26]:
sim.add_models(models_for_ensemble[1:], disable_safe_load=True)
for model in models_for_ensemble:
    # Copy to prevent changes before adding enzyme modules
    model = model.copy()
    # Attempt to determine steady state
    conc_sol, flux_sol = sim.simulate(
        models=model,
        time=(0, tfinal)
    )
    df = conc_sol.to_frame()
    if all(abs(df.iloc[-1, :] - df.iloc[-2, :]) < 1e-8):
        model.update_initial_conditions({k: v for k, v in df.iloc[-1, :].iteritems()})
        # Save a reference MASS model
        save_mass_json_model(
            mass_model=model,
            filename=f"./models/mass/without_enzymes/{model.id}.json")
        print(f"Saving {model.id}")
    else:
        print(f"No steady state for {model.id}.")

Saving Glycolysis_C0
Saving Glycolysis_C1
Saving Glycolysis_C2
Saving Glycolysis_C3
Saving Glycolysis_C4
Saving Glycolysis_C5
Saving Glycolysis_C6
Saving Glycolysis_C7
Saving Glycolysis_C8
Saving Glycolysis_C9


## Create Enzyme Modules

Assume 90% of flux goes through the major isozyme, the remaining through the minor isozyme

In [27]:
isozyme1_percent = 0.9
isozyme2_percent = 0.1

# Isozymes and flux split percentages,
isozymes_and_flux_splits = {
    "PFK": {
        "PFK1": isozyme1_percent,
        "PFK2": isozyme2_percent,
    },
    "FBP": {
        "FBP1": isozyme1_percent,
        "FBP2": isozyme2_percent,
    },
    "FBA": {
        "FBA1": isozyme1_percent,
        "FBA2": isozyme2_percent,
    },
    "PGM": {
        "PGMi": isozyme1_percent,
        "PGMd": isozyme2_percent,
    },
    "PYK": {
        "PYK1": isozyme1_percent,
        "PYK2": isozyme2_percent,
    },
}
        
isozymes_and_flux_splits

{'PFK': {'PFK1': 0.9, 'PFK2': 0.1},
 'FBP': {'FBP1': 0.9, 'FBP2': 0.1},
 'FBA': {'FBA1': 0.9, 'FBA2': 0.1},
 'PGM': {'PGMi': 0.9, 'PGMd': 0.1},
 'PYK': {'PYK1': 0.9, 'PYK2': 0.1}}

In [28]:
from module4_functions import make_enzyme_module_from_dir

In [29]:
final_ensemble = []
for model in models_for_ensemble:
    enzyme_modules = {}
    for reaction in model.reactions.get_by_any(reaction_list):
        # PGM & PGK needs flux flipped since enzyme module stoichiometry
        # is reversed compared to lone reaction.
        if reaction.id in ["PGK", "PGM"]:
            flux = -reaction.steady_state_flux
        else:
            flux = reaction.steady_state_flux
        # Make isozymes
        if reaction.id in isozymes_and_flux_splits:
            isozymes_and_flux_split = isozymes_and_flux_splits[reaction.id]
            isozyme_modules = []
            for isozyme, flux_split in isozymes_and_flux_split.items():
                enzyme_module = make_enzyme_module_from_dir(
                    enzyme_id=isozyme,
                    steady_state_flux=flux * flux_split, # Split flux for isozymes
                    metabolite_concentrations=model.initial_conditions,
                    path_to_dir="./data/enzyme_module_data",
                    kcluster=1,
                    enzyme_gpr=reaction.gene_reaction_rule,
                    zero_tol=1e-10)
                isozyme_modules += [enzyme_module]
            enzyme_modules[reaction] = isozyme_modules
        else:
            enzyme_module = make_enzyme_module_from_dir(
                enzyme_id=reaction.id,
                steady_state_flux=flux,
                metabolite_concentrations=new_model.initial_conditions,
                path_to_dir="./data/enzyme_module_data",
                kcluster=1,
                enzyme_gpr=reaction.gene_reaction_rule,
                zero_tol=1e-10)
            enzyme_modules[reaction] = [enzyme_module]
    for reaction_to_remove, enzymes_to_add in enzyme_modules.items():
        model.remove_reactions([reaction_to_remove])
        for enzyme in enzymes_to_add:
            model = model.merge(enzyme, inplace=True)
    final_ensemble += [model]
    print(f"Finished {model.id}")

Finished Glycolysis_C0
Finished Glycolysis_C1
Finished Glycolysis_C2
Finished Glycolysis_C3
Finished Glycolysis_C4
Finished Glycolysis_C5
Finished Glycolysis_C6
Finished Glycolysis_C7
Finished Glycolysis_C8
Finished Glycolysis_C9


# Inspect a model

In [30]:
reference_model = final_ensemble[0]
sim = Simulation(reference_model)
sim.integrator.maximum_time_step = 100
sim.integrator.absolute_tolerance = 1e-15
sim.integrator.relative_tolerance = 1e-9

tfinal = 1e5
conc_sol, flux_sol = sim.simulate(reference_model, time=(0, tfinal))

In [31]:
to_observe = {}
for enz_module in reference_model.enzyme_modules:
    # Save list of observables enzyme forms per enzyme
    to_observe[enz_module.id] = {
        "forms": [e.id for e in enz_module.enzyme_module_forms],
        "reactions": [e.id for e in enz_module.enzyme_module_reactions]
    }

In [32]:
plot_time_profile(conc_sol, observable=to_observe["PGI"]["forms"],
                  xscale="log", yscale="log", ylim=[1e-6, 1e-3])

alt.Chart(...)

## Simulate and save steady state models

In [33]:
sim.add_models(final_ensemble[1:], disable_safe_load=True)
for model in final_ensemble:
    # Copy to prevent changes before adding enzyme modules
    # Attempt to determine steady state
    conc_sol, flux_sol = sim.simulate(
        models=model,
        time=(0, tfinal)
    )
    df = conc_sol.to_frame()
    # Check if steady state reached by comparing final time points
    if all(abs(df.iloc[-1, :] - df.iloc[-2, :]) < 1e-8):
        model.update_initial_conditions({k: v for k, v in df.iloc[-1, :].iteritems()})
        # Save a reference MASS model
        save_mass_json_model(
            mass_model=model,
            filename=f"./models/mass/with_enzymes/{model.id}.json")
        print(f"Saving {model.id}")
    else:
        print(f"No steady state for {model.id}.")

Saving Glycolysis_C0
Saving Glycolysis_C1
Saving Glycolysis_C2
Saving Glycolysis_C3
Saving Glycolysis_C4
Saving Glycolysis_C5
Saving Glycolysis_C6
Saving Glycolysis_C7
Saving Glycolysis_C8
Saving Glycolysis_C9
